# Wheat Disease Detection on Leaf Images using EfficientNetB0

## Import Data from Google Drive Folder

In [ ]:
!conda install -y gdown 
import gdown 

In [ ]:
url = 'https://drive.google.com/uc?id=1rdbF95HfP4K_lznMirByqvkYzEzaavg-'
output = 'input.zip'
gdown.download(url, output)

In [ ]:
!unzip input.zip

In [ ]:
!pip install imutils

In [ ]:
rm input.zip

## Libraries and Imports

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import AveragePooling2D, Dense, Dropout, Flatten, Input
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from tensorflow.keras.applications import EfficientNetB0
from imutils import paths
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pickle

## Data Preprocessing

## Extract GLCM and GLCM Features

In [ ]:
#! /usr/bin/python
# - *- coding: UTF-8-*-
import cv2
import math
from numpy.linalg import matrix_power

# Define the maximum number of gray levels
gray_level =128

def maxGrayLevel(img):
    max_gray_level=0
    (height,width)=img.shape
    #print("The height and width of the image are: height,width",height,width)
    for y in range(height):
        for x in range(width):
            if img[y][x]> max_gray_level:
                max_gray_level = img[y][x]
                #print("max_gray_level:",max_gray_level)
    return max_gray_level+1

def getGlcm(input,d_x,d_y):
    srcdata=input.copy()
    ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
    (height,width)= input.shape

    max_gray_level=maxGrayLevel(input)
    # If the number of gray levels is greater than gray_level, the gray level of the image is reduced to gray_level, reduce the size of the gray-level co-occurrence matrix
    if max_gray_level > gray_level:
        for j in range(height):
            for i in range(width):
                srcdata[j][i]= srcdata[j][i]*gray_level / max_gray_level

    for j in range(height-d_y):
        for i in range(width-d_x):
            rows = srcdata[j][i]
            cols = srcdata[j + d_y][i+d_x]
            ret[rows][cols]+=1.0
    
    for i in range(gray_level):
        for j in range(gray_level):
            ret[i][j]/=float(height*width)
    return ret

def feature_computer(p):
    # con: Contrast 
    # reflects the sharpness of the image and the depth of the grooves of the texture. 
    # The sharper the texture, the greater the contrast, the greater the contrast.
    
    # ent: Entropy 
    # measures the randomness of the amount of information contained in the image 
    # and expresses the complexity of the image. When all the values in the co-occurrence matrix 
    # are equal or the pixel values show the greatest randomness, the entropy is the largest.
    
    # asm: (Energy) Angle second-order moment
    # a measure of the uniformity of image gray distribution and texture thickness. 
    # When the image texture is uniform and regular, the energy value is large; 
    # on the contrary, the element values of the gray-level co-occurrence matrix are similar, and the energy value is small.
    
    # idm: (Homogenity) The inverse difference matrix 
    # is also called the inverse variance, which reflects the clarity and regularity of the texture. 
    # The texture is clear, regular, easy to describe, and has a larger value.
    
    # cor: Correlation Feature
    # 
    # 
    
    # shd: Shade Feature
    #
    #
    
    # prm: Prominence
    # 
    #
    
    Con=0.0
    Enp=0.0
    Asm=0.0
    Idm=0.0
    Cor=0.0
    Shd=0.0
    Cor=0.0
    Prm=0.0
    meean=0.0
    
    
#     for i in range(gray_level):
#         for j in range(gray_level):
#             meean=
#     Cor=corrcoef(p)
#     A=
#     Shd=
    
    
    for i in range(gray_level):
        for j in range(gray_level):
            Con+=(i-j)*(i-j)*p[i][j]
            Asm+=p[i][j]*p[i][j]
            Idm+=p[i][j]/(1+(i-j)*(i-j))
            
            if p[i][j]>0.0:
                Enp+=p[i][j]*math.log(p[i][j])
    return Asm,Con,-Enp,Idm

def test_glcm(image_name):
    img = cv2.imread(image_name)
    try:
        img_shape=img.shape
    except:
        print('imread error')
        return

    # If you use &#39;/&#39;Will report TypeError: integer argument expected, got float
    # In fact, the main error is because of cv2.The parameter in resize is required to be an integer
    img=cv2.resize(img,(img_shape[1]//2,img_shape[0]//2),interpolation=cv2.INTER_CUBIC)

    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    glcm_0=getGlcm(img_gray,1,0)
    # glcm_1=getGlcm(src_gray,0,1)
    # glcm_2=getGlcm(src_gray,1,1)
    # glcm_3=getGlcm(src_gray,-1,1)print(glcm_0)

    #asm,con,eng,idm=feature_computer(glcm_0)
    #return [asm,con,eng,idm]
    return glcm_0

In [ ]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])
imagePaths = list(paths.list_images('../input/wheatdiseasedetectionlwdcd/Large Wheat Disease Classification Dataset'))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    print(imagePath, " -->", label)
    # if the label of the current image is not part of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
 
    #     image = cv2.imread(imagePath)
    #     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #     image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    # [asm,con,eng,idm]=test_glcm(imagePath)
    glcm=test_glcm(imagePath)
    data.append(glcm)
    labels.append(label)

In [ ]:
len(data)

In [ ]:
# GLCM Features Extracted

# import pandas as pd
# df1=pd.DataFrame(data, columns=["Contrast", "Entropy", "Energy", "Homogenity"])
# df2=pd.DataFrame(labels, columns=["Classes"])
# df = pd.merge(df1, df2, right_index=True, left_index=True)
# df.head()

In [ ]:
data[0].shape

In [ ]:
data[1].shape

In [ ]:
for i in range(0, len(df['Classes'])):
    if df['Classes'].iloc[i]=='Crown and Root Rot':
        df['Classes'][i]=0
    elif df['Classes'].iloc[i]=='Healthy Wheat':
        df['Classes'][i]=1
    elif df['Classes'].iloc[i]=='Leaf Rust':
        df['Classes'][i]=2
    else :
        df['Classes'][i]=3

In [ ]:
df.sort_values('Classes')

In [ ]:
df.to_csv('Features.csv')

## Data Labelling and Splitting the Data

In [ ]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
 test_size=0.25, stratify=labels, random_state=42)

## Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
 rotation_range=30,
 zoom_range=0.15,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.15,
 horizontal_flip=True,
 fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean


## Model Definition

In [ ]:
# load the EfficientNet-B0, ensuring the head FC layer sets are left ff

headmodel = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(128,128,3)),
    classes=1000,
    classifier_activation="relu",
)
# construct the head of the model that will be placed on top of the
# the base model
model = headmodel.output
model = AveragePooling2D(pool_size=(5, 5))(model)
model = Flatten(name="flatten")(model)
model = Dense(512, activation="relu")(model)
model = Dropout(0.4)(model)
model = Dense(len(lb.classes_), activation="softmax")(model)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
moodel = Model(inputs=headmodel.input, outputs=model)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in headmodel.layers:
    layer.trainable = False

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(moodel, to_file='model_EffNetB0.png', show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(moodel, to_file='model_Extended.png')

## Training Model via Transfer Learning

In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
opt = Adam(learning_rate=1e-3)
moodel.compile(loss="categorical_crossentropy", optimizer=opt,
               metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
H = moodel.fit(
    trainAug.flow(trainX, trainY, batch_size=64),
    steps_per_epoch=len(trainX) // 64,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 64,
    epochs=20)

## Extending the Epoches

In [ ]:
H1 = moodel.fit(
    trainAug.flow(trainX, trainY, batch_size=64),
    steps_per_epoch=len(trainX) // 64,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 64,
    epochs=10)

In [ ]:
# evaluate the network
predictions = moodel.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = 20
plt.plot(np.arange(0, N), H.history['accuracy'], label="Training Accuracy")
plt.plot(np.arange(0, N), H.history['val_accuracy'], label="Test Accuracy")
plt.title('EfficientNetB0 Model Train vs Test Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()
plt.savefig(".\Accuracy_Plot_EffNetB0.png")
plt.plot(H.history['loss'], label="Training Loss")
plt.plot(H.history['val_loss'], label="Test Loss")
plt.title('EfficientNetB0 Model Train vs Test Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()
plt.savefig(".\Loss_Plot_EffNetB0.png")

In [ ]:
N = 10
plt.plot(np.arange(0, N), H1.history['accuracy'], label="Training Accuracy")
plt.plot(np.arange(0, N), H1.history['val_accuracy'], label="Test Accuracy")
plt.title('EfficientNetB0 Model Train vs Test Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.show()
plt.savefig(".\Accuracy_Plot_EffNetB0.png")
plt.plot(H1.history['loss'], label="Training Loss")
plt.plot(H1.history['val_loss'], label="Test Loss")
plt.title('EfficientNetB0 Model Train vs Test Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()
plt.savefig(".\Loss_Plot_EffNetB0.png")

### From both the above images, it seems 26 epoch is the best suitable choice.. after 26, it starts overfitting